# Sensor Data Fusion

Lecturer: Prof. Baum \
Tutor: Shishan Yang and Laura Wolf \
Semester: Winter 20/21

## Homework 3

Look again at the GPS setting of last homework (sattelites are $20200 \text{km}$ above mean sea level, each  satellite broadcasts its location in spherical coordinates $[\theta, \phi, r]^T$ plus the emission time (see figures below), assume that the speed-of-light is $c=3\cdot10^8\text{ms}^{-1}$ and the Earth is an ideal sphere with radius $r^{\text{E}}=6370 \text{km}$)

This time, the device only receives the first three sattelite signals at time $t=0\text{s}$:

| $i$ | $p_i$ | $t_i$ |
| :--- | :---: | ---: |
| 1 | $[  0^{°},  40^{°}, 20200 \text{km}]^T$ | -67.603 ms |
| 2 | $[ 10^{°},  20^{°}, 20200 \text{km}]^T$ | -70.102 ms |
| 3 | $[ 10^{°}, -10^{°}, 20200 \text{km}]^T$ | -78.690 ms |


![Satellite Visuals](https://owncloud.gwdg.de/index.php/s/guLnIkLe4aOcL1E/download)

---
The following tasks will have missing sections marked that you should fill out. 

Missing code parts are marked by
```
# ... code code code
=== YOUR CODE HERE ===

=== END OF YOUR CODE ===
# ... code code code
```
If you are asked to implement a function, make sure to check what variable will be returned by the function and to fill it accordingly. Do not change code outside of the indicated sections.

Furthermore, some questions require theoretical answers instead of python code.

Such questions will have a field marked like this: 

=== YOUR ANSWER HERE === 

In [2]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.linalg import inv
import sympy as sym

In [3]:
# Definition of global variables
speed_light = 3 * math.pow(10, 5)  # Speed of light def. - 10^5 because we want km/s
R = 6370  # earth radius (assuming earth is a perfect sphere)
t = np.array([67.603e-3, 70.102e-3, 78.690e-3])  # emission times

# location in spherical coordinates
p = [
    [0, 40, 20200],
    [10, 20, 20200],
    [10, -10, 20200]
    ]
p = np.array(p)
spher_coor_p = p  # add a second alias...

# Ground Truth target location from the last exercise
gt = np.array([4390.2170784, 589.75165586, 4584.055838])

In [4]:
# You will also be provided with sphere_to_cartesian and cartesian_to_sphere functions
def sphere_to_cartesian(p_sphere, R):
    """
    Function that converts spherical to cartesian coordinates. 
    :param p_sphere: 3x1 vector [theta, phi, R] with
    theta, longitude in degree
    phi, latitude in degree
    r, altitude in km
    :param R: earth radius in km
    :return: p_cartesian, converted cartesian coordinates, as 3x1 vector [x,y,z]
    """
    # make sure everything is numpy
    p_sphere = np.array(p_sphere)
    
    # unbox p_sphere and convert to radians
    theta = math.radians(p_sphere[0])
    phi = math.radians(p_sphere[1])
    r = p_sphere[2]
    
    # calculate x,y,z, position
    x = (R + r) * math.cos(phi) * math.cos(theta);
    y = (R + r) * math.cos(phi) * math.sin(theta);
    z = (R + r) * math.sin(phi); 
    p_cartesian = [x, y, z]
    
    return p_cartesian

def cartesian_to_sphere(p_cartesian, R):
    """
    :param p_cartesian: Point represented in cartesian coordinates as 3x1 vector
    :param R: Earth Radius in km
    :return: p_sphere, point represented in sphere coordinates, 
            3x1 vector: [theta, phi,r],  with
            theta, longitude in degree
            phi, latitude in degree
            r, altitude in km
    """
    p_cartesian = np.array(p_cartesian)
    
    x,y,z = p_cartesian  # unpack array
    r = math.sqrt(x**2 + y**2 + z**2) - R
    theta = math.degrees(np.arctan(y/x))
    phi = math.degrees(np.arctan(z / (math.sqrt(x**2 + y**2))))
    p_sphere = [theta, phi, r]
    
    return p_sphere

In [5]:
# Further variable definitions:

cart_coor_p = np.array([sphere_to_cartesian(pos, R) for pos in spher_coor_p])

d = speed_light * t

---
### a)
Would the reformulation from last time still work? If not, explain why.

=== YOUR ANSWER HERE ===

---
### b)
Instead of reformulating the measurement equation into a linear equation, implement the Gauss-Newton method to calculate the least squares solution.

Hints: 
- you will need to use symbolic python (the package sympy is already imported)
- you will need to use sym.Matrix instead of np.array for most variables: convert np arrays into sympy as necessary, the two packages don't mix well
- sympy provides .jacobian(...) for calculating the Jacobian
- sympy provides .subs(...) for symbolic substitution 


In [6]:
def gauss_newton(y, h, x1, x2, x3, x_init):
    """
    Applies the Gauss-Newton method.
    :param y: measurements
    :param h: non-linear measurement matrix
    :param x1: symbolic variable representing the dimension of the state
    :param x2: symbolic variable representing the dimension of the state
    :param x3: symbolic variable representing the dimension of the state
    :return: x_hat, the resulting estimate
    """
    y = sym.Matrix(3, 1, d)
    x_init = np.array(x_init)

    # === YOUR CODE HERE ===
    x_hat = sym.Matrix(3, 1, x_init)

    update = 100
    iteration_cnt = 0
    while update >= 1:

        r_x_hat = y - h.subs({
            x1: x_hat[0],
            x2: x_hat[1],
            x3: x_hat[2]
        })

        e = sym.Matrix(y - h)
        j = e.jacobian(x)
        j_sub = j.subs({
            x1: x_hat[0],
            x2: x_hat[1],
            x3: x_hat[2]
        })

        next_x_hat = x_hat - (j_sub.T @ j_sub).inv() @ j_sub.T @ r_x_hat
        update = sym.Matrix.norm(next_x_hat - x_hat)
        x_hat = next_x_hat
        iteration_cnt += 1

        print("iteration: ", iteration_cnt)
        print("x_hat: ", x_hat)
        print("update: ", update)
        print()


    # === END OF YOUR CODE ===

    return x_hat

In [7]:
# First, we create the symbolic variables
x1 = sym.symbols("x1")
x2 = sym.symbols("x2")
x3 = sym.symbols("x3")

x = sym.Matrix(3, 1, [x1, x2, x3])

# FYI: http://omz-software.com/pythonista/sympy/modules/matrices/matrices.html
# if you want a deeper understanding of why lambda functions are used in the def. of h
h = [
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[0,j] - x[j,0])**2).T)),
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[1,j] - x[j,0])**2).T)),
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[2,j] - x[j,0])**2).T))
]

h = sym.Matrix(3, 1, h)

# Next, we apply the gauss newton algorithm
x_init = np.array([0, 0, 0])
x_hat_gn = gauss_newton(d,h,x1,x2,x3,x_init)
x_hat_gn = np.reshape(np.array(x_hat_gn), (3,)).astype(float)

print("\n---\nGN Solution after using {} as initial guess:".format(x_init))
print(x_hat_gn)
print("\nDistance to true solution: {:.4f}".format(np.linalg.norm(gt-x_hat_gn)))

iteration:  1
x_hat:  Matrix([[3727.47611907220], [1611.17394154732], [5341.86966329438]])
update:  6710.10664503922

iteration:  2
x_hat:  Matrix([[4344.67777511136], [607.903745791851], [4563.74732858568]])
update:  1411.72353443627

iteration:  3
x_hat:  Matrix([[4389.86711202113], [590.076772002260], [4584.07020280589]])
update:  52.6582983148003

iteration:  4
x_hat:  Matrix([[4389.90660495346], [590.012901048791], [4584.03305661328]])
update:  0.0837796516262482


---
GN Solution after using [0 0 0] as initial guess:
[4389.90660495  590.01290105 4584.03305661]

Distance to true solution: 0.4064


---
### c)
Alternatively, use the Bancroft solution to determine the position of the GPS device.

Hint: 
- numpy provides np.roots() to find the roots of a polynomial

__Solution (Theory):__
$\newcommand\vect[1]{\begin{bmatrix}#1\end{bmatrix}}$

Squared meas. equation:

$
d_i^2 = ||x - p_i ||^2+ e^*_i$

$=  (x_1-p_{i,1})^2  + (x_2-p_{i,2})^2 + e^*_i$

$=   -2x_1 p_{i,1}  - 2x_2 p_{i,2}  + ||p_i||^2 +R^2   + e^*_i$

with   $R^2:= ||x||^2= (x_1)^2 + (x_2)^2$


Linear measurement equation for given $R^2$:
$$y =  \textbf{H}_1 x   + \textbf{H}_2 R^2 + e^* $$
with
$$
y  =  \vect{d_1^2 - ||p_1 ||^2 \\ \vdots \\ d_m^2 - ||p_m ||^2 } \enspace, \enspace   \textbf{H}_1  =  \vect{ -2 p_{i,1}  & -2 p_{i,2}      \\  \vdots & \vdots  \\   -2 p_{m,1}  & -2 p_{m,2} } \enspace , \enspace  \textbf{H}_2  =  \vect{ 1      \\  \vdots \\   1}
$$



Least squares solution for a fixed $R^2$:

\begin{eqnarray*}
x^{LS}(R^2)  &=& (\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T}  (y- \textbf{H}_2 R^2) \\  
                &= &  z_1 + R^2   z_2 
\end{eqnarray*}

with $z_1 := (\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T} y$ and $z_2:= -(\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T}   \textbf{H}_2 $ 


What is $R^2$? 

\begin{eqnarray*}
R^2 &=& ||x^{LS}(R^2)||^2 \\
    & =&  (z_1 + R^2 z_2)^\text{T}  \cdot (z_1 + R^2 z_2)  
\end{eqnarray*}


Solve the following quadratic equation for $R^2$:

\begin{equation*}
0 = z_1^\text{T}  z_1  + z_1^\text{T}  z_2  R^2  + R^2 z_2^\text{T}  z_1  +  (R^2)^2 z_2^\text{T}  z_2   -R^2 
\end{equation*}


In [128]:
def bancroft(p, d):
    """
    Calculates the bancroft solution.
    :param p: The cartesian coordinates with shape (3x3)
    :param d: The distances between satellites and receiver with shape (3,)
    :return: x_hat, the position estimate
    """
    # ensure everything is numpy
    p = np.array(p)
    d = np.array(d)
    
    # === YOUR CODE HERE ===
    p_squared_norms = np.square([np.linalg.norm(arr) for arr in p])
    d_squared_dist = np.square(d)
    y = np.array([d_squared_dist - p_squared_norms])

    H_1 = -2 * p
    H_2 = np.ones((3, 1))

    z_1 = np.linalg.inv(H_1.T @ H_1) @ H_1.T @ y.T
    z_2 = - np.linalg.inv(H_1.T @ H_1) @ H_1.T @ H_2

    print(1, 3)
    print("z_2: ", z_2)

    polylnomial_1 = np.array(z_2.T @ z_2).item()
    polylnomial_2 = np.array(z_1.T @ z_2 + z_2.T @ z_1 - 1).item()
    polylnomial_3 = np.array(z_1.T @ z_1).item()

    coef = np.array([polylnomial_1, polylnomial_2, polylnomial_3])
    print("coef: ", coef)

    res = np.roots(coef)
    print("res: ", res)

    a = np.argmin(np.abs(res))
    x_hat = z_1 + res[a] * z_2
    x_hat = x_hat.T[0]

    # === END OF YOUR CODE ===

    return x_hat

In [129]:
# Now, the bancroft solution will be applied to find an estimate
print("old: ", cart_coor_p)
print("cart_coor_p: ", )
x_hat_b = bancroft(cart_coor_p, d)
print("Bancroft solution:")
print(x_hat_b)
print("\nDistance to true solution: {:.4f}".format(np.linalg.norm(gt-x_hat_b)))

old:  [[20353.80085367     0.         17078.86678937]
 [24588.31848804  4335.58395969  9087.47520816]
 [25768.81646714  4543.73760408 -4613.83208061]]
cart_coor_p: 
1 3
z_2:  [[ 2.31406682e-05]
 [-1.94715186e-05]
 [ 1.69797261e-06]]
coef:  [ 9.17513670e-10 -8.78801240e-01  3.41932808e+07]
res:  [9.17174490e+08 4.06327543e+07]
Bancroft solution:
[4389.90660512  590.01290096 4584.03305667]

Distance to true solution: 0.4064
